# MA3831 A3 WebCrawler and NLP System

In [30]:
# Import Libraries
import os
import requests
import pandas as pd
import nltk
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import urllib.parse
import json
from pygments import highlight, lexers, formatters
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
import pickle

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import PorterStemmer


Chose the https://www.imdb.com/chart/top/?ref_=nv_mv_250&sort=user_rating%2Cdesc website to look through, instead of the default "ranking" they use. Unsure as to what that ranking is based off so using user rating for each movie in descending order for the top 250. 

In [35]:
# Functions for
def get_html(url):
    """Return html from given url."""
    page = requests.get(url)
    return page

def clean_html(page):
    """Clean html from website."""
    soup = BeautifulSoup(page.text, 'lxml')
    return soup

def print_html(soup):
    """Print html from website."""
    print(soup.prettify())

In [37]:
# Retrieve html from website
url = 'https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/'
webpage = get_html(url)

In [38]:
webpage = clean_html(webpage)

In [39]:
# Print html from webpage
print_html(webpage)

<!DOCTYPE html>
<html class="hitim" lang="en-US">
 <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <!-- OneTrust Cookies Consent Notice start for rottentomatoes.com -->
  <script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() { }
  </script>
  <!-- OneTrust Cookies Consent Notice end for rottentomatoes.com -->
  <!-- OneTrust IAB US Privacy (USP) -->
  <script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
  </script>
  <!-- OneTrust 

In [40]:
# retrieve movie titles from html
scraped_movies = webpage.find_all('a', class_='title')
print(scraped_movies)
len(scraped_movies) # Expected 300 movies

[<a class="title" href="https://www.rottentomatoes.com/m/la_confidential">L.A. Confidential</a>, <a class="title" href="https://www.rottentomatoes.com/m/the_godfather">The Godfather</a>, <a class="title" href="https://www.rottentomatoes.com/m/1003707-casablanca">Casablanca</a>, <a class="title" href="https://www.rottentomatoes.com/m/seven_samurai_1956">Seven Samurai</a>, <a class="title" href="https://www.rottentomatoes.com/m/parasite_2019">Parasite</a>, <a class="title" href="https://www.rottentomatoes.com/m/schindlers_list">Schindler's List</a>, <a class="title" href="https://www.rottentomatoes.com/m/top_gun_maverick">Top Gun: Maverick</a>, <a class="title" href="https://www.rottentomatoes.com/m/toy_story_2">Toy Story 2</a>, <a class="title" href="https://www.rottentomatoes.com/m/chinatown">Chinatown</a>, <a class="title" href="https://www.rottentomatoes.com/m/on_the_waterfront">On the Waterfront</a>, <a class="title" href="https://www.rottentomatoes.com/m/the_battle_of_algiers">The 

300

In [41]:
# Get movie titles
titles = []

for title in scraped_movies:
    titles.append(title.get_text().strip())
print(titles)
len(titles)

['L.A. Confidential', 'The Godfather', 'Casablanca', 'Seven Samurai', 'Parasite', "Schindler's List", 'Top Gun: Maverick', 'Toy Story 2', 'Chinatown', 'On the Waterfront', 'The Battle of Algiers', 'Toy Story', 'Rear Window', 'Modern Times', 'How to Train Your Dragon', 'All About Eve', 'Spirited Away', 'Up', 'The Third Man', 'Spotlight', 'Spider-Man: Into the Spider-Verse', 'The Philadelphia Story', 'Finding Nemo', "Singin' in the Rain", '12 Angry Men', 'Toy Story 3', 'Sunset Boulevard', 'Coco', 'The Godfather, Part II', 'Three Colors: Red', 'Selma', 'Zootopia', 'Citizen Kane', 'Annie Hall', 'Cool Hand Luke', 'The Holdovers', 'Inside Out', 'Dr. Strangelove Or: How I Learned to Stop Worrying and Love the Bomb', 'Let the Right One In', 'The Lord of the Rings: The Two Towers', 'Knives Out', 'M', 'Toy Story 4', 'The Wrestler', 'Goodfellas', 'The Wizard of Oz', 'Double Indemnity', 'Psycho', 'Paddington 2', 'Before Sunrise', 'The Dark Knight', 'The Maltese Falcon', 'It Happened One Night', 'T

300

In [42]:
# Get URLS for movies
urls = []

for link in scraped_movies:
    urls.append(link.get('href'))
print(urls)
len(urls)

['https://www.rottentomatoes.com/m/la_confidential', 'https://www.rottentomatoes.com/m/the_godfather', 'https://www.rottentomatoes.com/m/1003707-casablanca', 'https://www.rottentomatoes.com/m/seven_samurai_1956', 'https://www.rottentomatoes.com/m/parasite_2019', 'https://www.rottentomatoes.com/m/schindlers_list', 'https://www.rottentomatoes.com/m/top_gun_maverick', 'https://www.rottentomatoes.com/m/toy_story_2', 'https://www.rottentomatoes.com/m/chinatown', 'https://www.rottentomatoes.com/m/on_the_waterfront', 'https://www.rottentomatoes.com/m/the_battle_of_algiers', 'https://www.rottentomatoes.com/m/toy_story', 'https://www.rottentomatoes.com/m/1017289-rear_window', 'https://www.rottentomatoes.com/m/modern_times', 'https://www.rottentomatoes.com/m/how_to_train_your_dragon', 'https://www.rottentomatoes.com/m/1000626-all_about_eve', 'https://www.rottentomatoes.com/m/spirited_away', 'https://www.rottentomatoes.com/m/up', 'https://www.rottentomatoes.com/m/the_third_man', 'https://www.rott

300

Have scrapped through the Rotten Tomatoes website and have gotten movie titles and the urls for each movie. Next need to scrap through each of the urls for each movie and get movie details like, released date, genre, rating, synopsis, director, 
Didn't take reviews as it doesnt relate to the movie genre


In [44]:
# Scrap through urls
movie_url = 'https://www.rottentomatoes.com/m/la_confidential'

movie = get_html(movie_url)
movie = clean_html(movie)
print(movie)

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/" xmlns="http://www.w3.org/1999/xhtml">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
</script>
<script type="text/javascript">
                function OptanonWrapper() { }
            </script>
<script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
</script>
<script src="/assets/pizza-pie/javascripts/bundles/roma/rt-common.js?single"></script>
<meta content="text/html; charset=utf-8" http-equiv="

In [52]:
x = movie.find_all('div', class_='category-wrap')
print(x) 

[<div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Director</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/curtis_hanson">Curtis Hanson</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Producer</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/dan-kolsrud">Dan Kolsrud</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/david_l_wolper">David L. Wolper</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Screenwriter</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/curtis_hanson">Curtis Hanson</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/1062226-james_ellroy">James Ellroy</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/brian_helgeland">Brian Helgeland</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Distributor</rt-text>
</dt>
<dd

In [71]:
infos = []

for info in x:
    infos.append(info.get_text().strip().replace('\n', ' '))
print(infos)

['Director   Curtis Hanson', 'Producer   Dan Kolsrud,  David L. Wolper', 'Screenwriter   Curtis Hanson,  James Ellroy,  Brian Helgeland', 'Distributor   Warner Home Vídeo,  Warner Bros.', 'Production Co   Warner Brothers,  Regency Enterprises', 'Rating   R', 'Genre   Crime,  Drama', 'Original Language   English', 'Release Date (Theaters)   Sep 19, 1997, Original', 'Release Date (Streaming)   Dec 12, 2015', 'Box Office (Gross USA)   $64.6M', 'Runtime   2h 16m', 'Sound Mix   Surround,  DTS,  Dolby Digital', 'Aspect Ratio   35mm,  Scope (2.35:1)']


In [78]:
for info in infos:
    print(info)

Director   Curtis Hanson
Producer   Dan Kolsrud,  David L. Wolper
Screenwriter   Curtis Hanson,  James Ellroy,  Brian Helgeland
Distributor   Warner Home Vídeo,  Warner Bros.
Production Co   Warner Brothers,  Regency Enterprises
Rating   R
Genre   Crime,  Drama
Original Language   English
Release Date (Theaters)   Sep 19, 1997, Original
Release Date (Streaming)   Dec 12, 2015
Box Office (Gross USA)   $64.6M
Runtime   2h 16m
Sound Mix   Surround,  DTS,  Dolby Digital
Aspect Ratio   35mm,  Scope (2.35:1)


In [57]:
y = movie.find('div', class_='synopsis-wrap').text
print(y) # 


Synopsis
Three policemen, each with his own motives and obsessions, tackle the corruption surrounding an unsolved murder at a downtown Los Angeles coffee shop in the early 1950s. Detective Lieutenant Exley (Guy Pearce), the son of a murdered detective, is out to avenge his father's killing. The ex-partner of Officer White (Russell Crowe), implicated in a scandal rooted out by Exley, was one of the victims. Sergeant Vincennes (Kevin Spacey) feeds classified information to a tabloid magnate (Danny DeVito).



In [58]:
movie.find('div', class_='synopsis-wrap').text.strip()

"Synopsis\nThree policemen, each with his own motives and obsessions, tackle the corruption surrounding an unsolved murder at a downtown Los Angeles coffee shop in the early 1950s. Detective Lieutenant Exley (Guy Pearce), the son of a murdered detective, is out to avenge his father's killing. The ex-partner of Officer White (Russell Crowe), implicated in a scandal rooted out by Exley, was one of the victims. Sergeant Vincennes (Kevin Spacey) feeds classified information to a tabloid magnate (Danny DeVito)."

Receives all information. Need to remove html and just leave headers and content for each one. The synopsis is in a different class then the rest of the movie information. So 2 different find_all's were used and will need to be used per movie.

[<div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Director</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/curtis_hanson">Curtis Hanson</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Producer</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/dan-kolsrud">Dan Kolsrud</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/david_l_wolper">David L. Wolper</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Screenwriter</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/curtis_hanson">Curtis Hanson</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/1062226-james_ellroy">James Ellroy</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/brian_helgeland">Brian Helgeland</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Distributor</rt-text>
</dt>
<dd

# Task 1: Overview

# Task 2: Web scrapping

# Task 3: Data Wrangling

# Task 4: Machine Learning

### Text Cleaning Functions:

Justification of why sample was chosen

How does my topic relate to the real world. What are the real world applications? Proof of concept? 
Word2Vec TF-IDF
Functions for code

Word embedding
Ethics of webscrapper. Why chosen?

Beatiful soup why?

Improved Netflix recommendations from the model. 

In [ ]:
# Text cleaning functions

def remove_by_regex(texts, regexp):
    """Removes text by regexp from a list."""
    output_texts = regexp.sub("", texts)
    return output_texts


def remove_urls(input_text):
    """Remove urls from a string."""
    return remove_by_regex(input_text, re.compile(r"http.?://[^\s]+[\s]?"))


def text_preprocessing(uncleaned_text):
    """Preprocessing text by removing urls and punctuation."""
    review = re.sub('[^a-zA-Z]', ' ', uncleaned_text)
    review = review.lower()
    review = remove_urls(review)
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    return review 

### Visualisation Functions:

In [ ]:
# A Variety of Visualisation Functions:



